In [9]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv
import ssl

sys.path.append('..')
sys.path.append('../..')
sys.path.append('../../..')
ssl._create_default_https_context = ssl._create_stdlib_context
load_dotenv()

# Must restart the kernel if any of the following imported modules are changed
from helpers.focusmate import fm_sessions_data_to_df
from api.helpers.focusmate import fetch_all_focusmate_sessions, fetch_focusmate_profile
from api.helpers.request import get_access_token_from_db
from api.test.sample_data.sessions import sample_sessions

fm_api_profile_endpoint = os.getenv("NEXT_PUBLIC_FM_API_PROFILE_ENDPOINT")
fm_api_sessions_endpoint = os.getenv("NEXT_PUBLIC_FM_API_SESSIONS_ENDPOINT")
session_id = ""
local_timezone = "America/New_York"

In [10]:
access_token = get_access_token_from_db(session_id)

profile_data = fetch_focusmate_profile(
        fm_api_profile_endpoint, access_token).get("user")
local_timezone: str = profile_data.get("timeZone")

# Option A: use all sessions
# member_since: str = profile_data.get("memberSince")
# sessions_data = await fetch_all_focusmate_sessions(
#     fm_api_sessions_endpoint, access_token, member_since)

# Option B: use sample sessions
sessions_data = sample_sessions

all_sessions = fm_sessions_data_to_df(sessions_data, local_timezone)
sessions = all_sessions[all_sessions['completed'] == True].copy()

sessions_copy = sessions.copy()

sessions_copy['date'] = sessions_copy['start_time'].dt.date

sessions_copy.sort_values('date', inplace=True)

sessions_copy.head()

,session_id,duration,start_time,requested_at,joined_at,completed,session_title,partner_id,date
0,9b6140bd-f2e3-44ef-8903-26834373e6c8,3000000,2024-05-28 14:15:00,2024-05-30 14:12:15.120065,2024-05-30 14:12:33.261514,True,,b4bc6097-4040-4ad6-a532-b211e2a1b314,2024-05-28
1,9b6140bd-f2e3-44ef-8903-26834373e6c8,3000000,2024-05-31 14:15:00,2024-05-30 14:12:15.120065,2024-05-30 14:12:33.261514,True,,b4bc6097-4040-4ad6-a532-b211e2a1b314,2024-05-31
2,9b6140bd-f2e3-44ef-8903-26834373e6c8,3000000,2024-05-31 14:15:00,2024-05-30 14:12:15.120065,2024-05-30 14:12:33.261514,True,,b4bc6097-4040-4ad6-a532-b211e2a1b314,2024-05-31
3,9b6140bd-f2e3-44ef-8903-26834373e6c8,3000000,2024-06-01 14:15:00,2024-05-30 14:12:15.120065,2024-05-30 14:12:33.261514,True,,b4bc6097-4040-4ad6-a532-b211e2a1b314,2024-06-01
4,a6563e8b-9a87-4ba9-9dd8-20da30c2b502,3000000,2024-06-03 09:15:00,2024-05-30 09:05:47.092976,2024-05-30 09:13:52.917311,True,,70a2fedf-1079-4f0e-b893-8b26067a0bc2,2024-06-03


In [11]:
sessions_copy = sessions_copy[sessions_copy['start_time'].dt.weekday < 5]

sessions_copy['day_over_day_delta'] = sessions_copy['date'].diff().dt.days


def adjust_delta(row):
    delta = row['day_over_day_delta']
    if pd.isna(delta):
        return delta
    if delta > 1:
        prev_day = row['date'] - pd.Timedelta(days=delta)
        if prev_day.weekday() == 4 and row['date'].weekday() == 0:
            # Friday to Monday (delta of 3) should be considered consecutive
            return 1
    return delta

sessions_copy['day_over_day_delta'] = sessions_copy.apply(adjust_delta, axis=1)

sessions_copy['consecutive_day_group_id'] = (sessions_copy['day_over_day_delta'] != 1).cumsum()

streak_lengths = sessions_copy.groupby('consecutive_day_group_id').size()

longest_streak = streak_lengths.max()

longest_streak_id = streak_lengths.idxmax()
longest_streak_dates = sessions_copy.loc[sessions_copy['consecutive_day_group_id'] == longest_streak_id, 'date']
longest_streak_start_date = longest_streak_dates.min()
longest_streak_end_date = longest_streak_dates.max()

print(longest_streak, (str(longest_streak_start_date), str(longest_streak_end_date)))
sessions_copy.tail(50)

4 ('2024-05-31', '2024-06-05')


,session_id,duration,start_time,requested_at,joined_at,completed,session_title,partner_id,date,day_over_day_delta,consecutive_day_group_id
0,9b6140bd-f2e3-44ef-8903-26834373e6c8,3000000,2024-05-28 14:15:00,2024-05-30 14:12:15.120065,2024-05-30 14:12:33.261514,True,,b4bc6097-4040-4ad6-a532-b211e2a1b314,2024-05-28,NaN,1
1,9b6140bd-f2e3-44ef-8903-26834373e6c8,3000000,2024-05-31 14:15:00,2024-05-30 14:12:15.120065,2024-05-30 14:12:33.261514,True,,b4bc6097-4040-4ad6-a532-b211e2a1b314,2024-05-31,3.0,2
2,9b6140bd-f2e3-44ef-8903-26834373e6c8,3000000,2024-05-31 14:15:00,2024-05-30 14:12:15.120065,2024-05-30 14:12:33.261514,True,,b4bc6097-4040-4ad6-a532-b211e2a1b314,2024-05-31,0.0,3
4,a6563e8b-9a87-4ba9-9dd8-20da30c2b502,3000000,2024-06-03 09:15:00,2024-05-30 09:05:47.092976,2024-05-30 09:13:52.917311,True,,70a2fedf-1079-4f0e-b893-8b26067a0bc2,2024-06-03,1.0,3
5,0e98d262-d3fd-4cca-9a58-47a239f8e639,3000000,2024-06-04 08:15:00,2024-05-26 10:33:07.137416,2024-05-30 08:12:48.786790,True,,fde6e2d1-df8e-4cc7-86fd-4f5a7a9f61b6,2024-06-04,1.0,3
6,dc5257dc-7e68-4ff9-a1c7-652af49216a8,3000000,2024-06-05 07:15:00,2024-05-26 10:33:03.998848,2024-05-30 07:14:57.723435,True,,70a2fedf-1079-4f0e-b893-8b26067a0bc2,2024-06-05,1.0,3
7,1e02bca9-cbdc-4d3d-a6ed-59a764e121d1,3000000,2024-06-07 08:15:00,2024-05-26 10:33:06.061438,2024-05-29 08:16:02.665237,True,,62c5fa49-4494-4ef0-b2d7-b07de37d27b8,2024-06-07,2.0,4
